# 專業 SEO 標題產生器

<p>此 Colab 檔案演示了如何在 Google Colab 環境中，透過較穩定的 Python 背景執行方法來啟動 Ollama 伺服器，並結合 LangChain 實現一個「專業SEO標題產生器」AI，專門生成符合 SEO 規範的網頁標題。</p>

<p><b>執行步驟：</b></p>
<ul>
    <li>請<b>務必依照儲存格的順序</b>，點擊每個儲存格左側的「播放」按鈕來執行。</li>
    <li><b>儲存格 3</b> 會需要一些時間來下載 Ollama 並啟動伺服器，請耐心等候其顯示「Ollama Server has been started」。</li>
    <li><b>儲存格 4</b> 會下載語言模型，第一次執行會花費較多時間。</li>
</ul>

<p><i>此筆記本中在 Colab 運行 Ollama 的穩定方法參考了 <a href='https://www.myapollo.com.tw/blog/google-colab-ollama/'>My APOLLO 的教學文章</a>。</i></p>

In [9]:
# 首先，安裝 LangChain 相關的 Python 套件
!pip install langchain langchain_community -q

In [ ]:
%%bash
# 使用 curl 下載 Ollama 的 Linux 執行檔
curl -fsSL https://github.com/ollama/ollama/releases/download/v0.1.33/ollama-linux-amd64 -o /usr/bin/ollama
# 檢查下載是否成功
if [ $? -ne 0 ]; then
    echo "Error: Failed to download Ollama binary."
    exit 1
fi
# 賦予執行權限
chmod +x /usr/bin/ollama
# 檢查權限賦予是否成功
if [ $? -ne 0 ]; then
    echo "Error: Failed to set execute permissions for Ollama binary."
    exit 1
fi
echo "Ollama binary downloaded and permissions set."

In [ ]:
import subprocess
import time
import requests
import os
import signal

ollama_log_file = "ollama_serve.log"

# 停止任何正在執行的 Ollama 伺服器程序
def stop_ollama_serve():
    try:
        # 查找 ollama serve 程序 ID
        # 使用 pgrep 搭配 -f 搜尋完整命令列
        pid_output = subprocess.check_output(["pgrep", "-f", "ollama serve"]).decode().strip()
        pids = pid_output.split('\n') if pid_output else []

        for pid in pids:
            if pid:
                print(f"找到正在運行的 Ollama 程序，PID：{pid}。嘗試停止它。")
                try:
                    # 發送終止信號
                    os.kill(int(pid), signal.SIGTERM)
                    # 等待一段時間讓其停止
                    time.sleep(2)
                    # 檢查是否仍在運行並在必要時強制終止
                    try:
                        os.kill(int(pid), 0) # 檢查程序是否存在
                        print(f"Ollama 程序 {pid} 未停止。正在強制終止。")
                        os.kill(int(pid), signal.SIGKILL)
                    except OSError:
                        print(f"Ollama 程序 {pid} 已成功停止。")
                except ProcessLookupError:
                    print(f"未找到 PID 為 {pid} 的程序，可能已經停止。")
                except Exception as e:
                    print(f"與程序 {pid} 交互時發生錯誤：{e}")

    except subprocess.CalledProcessError:
        print("未找到正在運行的 Ollama serve 程序。")
    except Exception as e:
        print(f"查找 Ollama 程序時發生錯誤：{e}")


# 定義一個函數，使用 nohup 在背景啟動 Ollama 伺服器
def run_ollama_serve_nohup():
    # 使用 nohup 在背景運行 ollama serve 命令，
    # 將輸出重定向到日誌檔案。
    command = ["nohup", "/usr/bin/ollama", "serve", ">", ollama_log_file, "2>&1", "&"]
    # 使用 subprocess.run 搭配 shell=True 執行命令
    # 這對於正確解釋重定向和背景符號是必要的
    subprocess.run(" ".join(command), shell=True, check=True)

print("正在停止任何現有的 Ollama 伺服器程序...")
stop_ollama_serve()

print("正在使用 nohup 在背景啟動 Ollama 伺服器...")

# 啟動 Ollama 伺服器
try:
    run_ollama_serve_nohup()
    print(f"Ollama 伺服器已啟動。正在檢查日誌檔案 '{ollama_log_file}' 的狀態...")
except subprocess.CalledProcessError as e:
    print(f"啟動 Ollama 伺服器時發生錯誤：{e}")
    print("嘗試檢查是否已經在運行...")

# 給伺服器一些時間啟動，然後檢查日誌檔案或 API
time.sleep(10) # 增加初始等待時間

ollama_ready = False
timeout = 180 # 增加超時時間至 3 分鐘
start_time = time.time()

while not ollama_ready and (time.time() - start_time) < timeout:
    # 檢查日誌檔案是否有伺服器正在運行的標誌
    if os.path.exists(ollama_log_file):
        with open(ollama_log_file, 'r') as f:
            log_content = f.read()
            if "Listening on" in log_content or "serving" in log_content.lower(): # 尋找伺服器正在服務的標誌
                 print("日誌檔案顯示 Ollama 伺服器正在運行。")
                 ollama_ready = True
                 break
    # 同時嘗試連接到 API
    try:
        response = requests.head("http://localhost:11434", timeout=5) # 增加超時時間
        if response.status_code == 200 or response.status_code == 404:
            print("Ollama API 可訪問。")
            ollama_ready = True
            break
    except requests.exceptions.ConnectionError:
        pass # 等待時預期的連接錯誤
    except requests.exceptions.ReadTimeout:
        print("檢查 API 時讀取超時，伺服器可能啟動緩慢。")
        pass # 讀取超時時繼續等待

    print("正在等待 Ollama 伺服器準備就緒...")
    time.sleep(5) # 增加檢查間隔時間

if ollama_ready:
    print("Ollama 伺服器已啟動並可訪問。")
else:
    print("錯誤：Ollama 伺服器在超時時間內未準備就緒。")
    if os.path.exists(ollama_log_file):
         print(f"{ollama_log_file} 的內容：")
         with open(ollama_log_file, 'r') as f:
             print(f.read())

In [ ]:
import time
import requests
import os

ollama_log_file = "ollama_serve.log"

# 在拉取模型之前等待 Ollama 伺服器準備就緒
ollama_ready = False
timeout = 180 # 將超時時間增加到 3 分鐘
start_time = time.time()

while not ollama_ready and (time.time() - start_time) < timeout:
    # 檢查日誌檔案是否有伺服器正在運行或出現錯誤的標誌
    if os.path.exists(ollama_log_file):
        with open(ollama_log_file, 'r') as f:
            log_content = f.read()
            if "Listening on" in log_content or "serving" in log_content.lower():
                 print("日誌檔案顯示 Ollama 伺服器正在運行。")
                 ollama_ready = True
                 break
            elif "error" in log_content.lower():
                 print(f"Ollama 日誌檔案中檢測到錯誤：{log_content}")
                 break # 如果在日誌中找到錯誤，則退出迴圈

    # 同時嘗試連接到 API
    try:
        response = requests.head("http://localhost:11434", timeout=5) # 增加超時時間
        if response.status_code == 200 or response.status_code == 404:
            print("Ollama API 可訪問。")
            ollama_ready = True
            break
    except requests.exceptions.ConnectionError:
        pass # 等待時預期的連接錯誤
    except requests.exceptions.ReadTimeout:
        print("檢查 API 時讀取超時，伺服器可能啟動緩慢。")
        pass # 讀取超時時繼續等待


    print("正在等待 Ollama 伺服器準備就緒...")
    time.sleep(5) # 增加檢查間隔時間

if ollama_ready:
    print("Ollama 伺服器已啟動且可訪問。正在拉取模型。")
    # 伺服器確認運行後，我們可以下載模型
    # 這個儲存格第一次執行時將花費幾分鐘下載模型檔案
    # 如果模型已存在，它會快速完成
    !ollama pull llama3
else:
    print("錯誤：Ollama 伺服器在超時時間內未準備就緒。無法拉取模型。")
    if os.path.exists(ollama_log_file):
         print(f"{ollama_log_file} 的內容：")
         with open(ollama_log_file, 'r') as f:
             print(f.read())

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama

# 初始化 LangChain 的 Ollama LLM，連接到我們在背景運行的伺服器
llm = Ollama(model="llama3")

# 創建詳細的 Prompt Template，賦予 AI 一個專業的 SEO 文案寫手 persona
template = """
你是一位擁有超過10年經驗的「專業SEO標題產生者」，專長是為部落格文章、新聞稿和網站頁面撰寫引人注目且高效的標題。

**你的任務是：**
根據我提供的「核心關鍵字」，生成5個符合以下技術和創意要求的標題：

**SEO 技術要求：**
1.  **關鍵字置前：** 盡可能將「核心關鍵字」放在標題的開頭。
2.  **長度適中：** 標題長度應介於 20 到 30 個中文字之間，以確保在搜尋結果頁面 (SERP) 上能完整顯示。
3.  **包含數字或疑問詞：** 適度地使用數字（例如「5個方法」）或疑問詞（例如「如何」、「為什麼」）可以有效提升點擊率。
4.  **避免關鍵字堆砌：** 標題需通順自然，不可惡意重複關鍵字。

**文案創意要求：**
1.  **吸引點擊 (High CTR)：** 標題需具備高度吸引力，能夠激發目標受眾的好奇心，讓他們想要點擊閱讀更多內容。
2.  **傳達價值：** 清楚地告訴讀者文章能為他們帶來什麼好處或解決什麼問題。
3.  **設定明確的目標受眾 (TA)：** 標題應能隱晦或明確地指向特定的讀者群體。

**輸出格式：**
請直接提供5個標題，每個標題一行，不要有任何額外的說明或編號。

---

**核心關鍵字：** {keyword}

**生成的5個標題：**
"""

# 創建 LangChain PromptTemplate
prompt = PromptTemplate(template=template, input_variables=["keyword"])

# 建立 LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

print("專業SEO標題產生器 LangChain Chain 已準備就緒！可以執行最後一個儲存格來生成標題。")

In [ ]:
import requests
import time

# 在繼續之前檢查 Ollama 伺服器是否正在運行
ollama_ready = False
timeout = 60 # 秒
start_time = time.time()

print("正在檢查 Ollama 伺服器是否可訪問...")
while not ollama_ready and (time.time() - start_time) < timeout:
    try:
        response = requests.head("http://localhost:11434", timeout=1)
        if response.status_code == 200 or response.status_code == 404:
            ollama_ready = True
            print("Ollama 伺服器可訪問。")
            break
    except requests.exceptions.ConnectionError:
        pass # 等待時預期的連接錯誤

    print("正在等待 Ollama 伺服器...")
    time.sleep(5)

if not ollama_ready:
    print("錯誤：Ollama 伺服器在超時時間內未準備就緒。")
    print("請重新運行「在背景啟動 Ollama 伺服器」儲存格，並等待它確認伺服器已準備就緒。")
else:
    # 現在，我們可以提供一個關鍵字給我們的「專業SEO標題產生器」AI 開始工作。
    # 您可以修改下面的 input_keyword 來生成不同主題的標題
    input_keyword = "數位孿生"

    print(f"正在為關鍵字「{input_keyword}」生成 SEO 標題...")
    print("-" * 40)

    # 運行 LLMChain 並傳入關鍵字
    # 確保 llm_chain 已在先前的儲存格 (setup_langchain_chain) 中定義
    if 'llm_chain' in locals():
        response = llm_chain.invoke(input_keyword)

        # 輸出最終結果
        print(response['text'])
    else:
        print("錯誤：LangChain chain (llm_chain) 未定義。")
        print("請先運行「設置 LangChain Chain」儲存格。")